## ✈️ Análisis de Clientes y Vuelos de la Aerolínea  

Objetivo  

**En este análisis exploraremos los datos de clientes inscritos en una membresía de aerolínea y sus vuelos.**

  **Realizaremos limpieza, visualización y evaluación de los datos para obtener insights relevantes.** 


In [ ]:
# Combinar más adelante los df
df_completo = df_loyalty.merge(df_flight, on="loyalty_number", how="inner")
# Esto combina la información del cliente con sus vuelos.
# Si hay clientes sin vuelos registrados, puedes usar how='left' en lugar de inner.

# Fase 1: Exploración y Limpieza

### 0. Importación de librerías

In [17]:
import pandas as pd
import numpy as np
# 📌 1. Configuración para gestión de nulos
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# 📌 2. Configurar pandas para ver todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 📌 3. Configuración para gráficos
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

In [31]:
# 1. Cargar el dataset de la información de los clientes sin la columna "Unnamed: 0"
df_loyalty = pd.read_csv("Customer_Loyalty_History.csv")
# Vista previa de los datos
display(df_loyalty.head())

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


**Carga y Exploración Inicial de Datos**

In [32]:
# 2. Cargar el dataset de la información de la actividad de los vuelos de los clientes sin la columna "Unnamed: 0"
df_flight = pd.read_csv("Customer_Flight_Activity.csv")
# Vista previa de los datos
display(df_flight.head())

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


## 1. Exploración Inicial:
- Realiza una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes
- Utiliza funciones de Pandas para obtener información sobre la estructura de los datos, la presencia de valores nulos y estadísticas básicas de las columnas involucradas.
- Une los dos conjuntos de datos de la forma más eficiente


*Comenzamos viendo la información de el csv de los datos de los clientes:*

In [33]:
print(f"Tamaño del dataset de clientes: {df_loyalty.shape}")

Tamaño del dataset de clientes: (16737, 16)


In [34]:
print(f"Tamaño del dataset de vuelos: {df_flight.shape}")

Tamaño del dataset de vuelos: (405624, 10)


In [35]:
# Información general de variables en dataset de clientes
df_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

In [36]:
# Información general de variables en dataset de vuelos
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


## 2. Limpieza de los datos
- Elimina o trata los valores nulos, si los hay, en las columnas clave para asegurar que los datos
estén completos.
- Verifica la consistencia y corrección de los datos para asegurarte de que los datos se
presenten de forma coherente.
- Realiza cualquier ajuste o conversión necesaria en las columnas (por ejemplo, cambiar tipos de datos) para garantizar la adecuación de los datos para el análisis estadístico

### Preparación de los datos

- Vamos a estandarizar los datos. Primero, vemos que los nombres de las columnas empiezan con mayúsculas, todos los nombres de las columnas se cambiarán a minúsculas. 
-Modificaremos los nombres de las variables de cada dataset, es decir, cambiaremos los nombres de las columnas eliminando espacios y comas.

- Haciendo el análisis exploratorio nos hemos dado cuenta de que algunas de las variables (`xxxx`, `oooo`) no son del tipo que deberían. Esto es debido a que los decimales están establecidos como comas y no con puntos. Crea una función que nos permita cambiar esas comas por puntos para que los datos tengan el tipo correcto. 

In [37]:
# Convertir nombres de columnas a minúsculas y reemplazar espacios por '_'
df_loyalty.columns = df_loyalty.columns.str.lower().str.replace(" ", "_")
df_flight.columns = df_flight.columns.str.lower().str.replace(" ", "_")
# Es importante también que renombremos la variable del id del cliente "Loyalty Number"
# Para ello:
df_loyalty.rename(columns={"Loyalty Number": "loyalty_number"}, inplace=True)
df_flight.rename(columns={"Loyalty Number": "loyalty_number"}, inplace=True)


In [38]:
df_loyalty.columns

Index(['loyalty_number', 'country', 'province', 'city', 'postal_code',
       'gender', 'education', 'salary', 'marital_status', 'loyalty_card',
       'clv', 'enrollment_type', 'enrollment_year', 'enrollment_month',
       'cancellation_year', 'cancellation_month'],
      dtype='object')

In [39]:
df_flight.columns

Index(['loyalty_number', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed'],
      dtype='object')

Notas:
**df_loyalty:** 
Análisis exploratorio
- Comparar year y month para ver cual eliminar
- Analizar patrones de compra según flights_booked (mediana ... hacer gráfica)

Limpieza
- Cambiar distance de int a float
- Renombrar columnas
- Gestión de valores nulos en annual_salary, cancellation_year y cancellation_month

Análisis estadístico
- ¿Afecta la distancia de los vuelos que se recorren a los puntos?
- ¿En qué año se han registrado más vuelos o más reservas?
- ¿Los clientes que viajan en grupo tienen más puntos acumulados o canjeados? Hipótesis

In [41]:
df_loyalty["education"].unique()

array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
      dtype=object)

In [ ]:
'''Verificar valores únicos en variables categóricas
Antes de seguir con la limpieza, verifica qué valores
 hay en las variables categóricas para ver si necesitan ajustes:'''

In [43]:
categoricas = ["gender", "education", "marital_status", "loyalty_card", "enrollment_type"]
for col in categoricas:
    print(f"Valores únicos en {col}: {df_loyalty[col].unique()}")


Valores únicos en gender: ['Female' 'Male']
Valores únicos en education: ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
Valores únicos en marital_status: ['Married' 'Divorced' 'Single']
Valores únicos en loyalty_card: ['Star' 'Aurora' 'Nova']
Valores únicos en enrollment_type: ['Standard' '2018 Promotion']


In [ ]:
df = df.rename(colums= {‘key’: ‘value’}, index= {‘key’: ‘value’}, inplace=True) 

In [ ]:
# Dict comprehension para unificar nombres de todas las columnas del DataFrame: 
# Modifica en todos los valores de la columna sustituyendo el . por espacio.
df.columns = df.columns.str.lower()
1. nuevas_columnas = {columna: columna.lower().replace(".", "") for columna 
in df.columns} 
2. df.rename(columns = nuevas_columnas, inplace = True) 
#Modifica todas las columnas poniendo sus nombres en minúsculas y sustituye el . 
#por nada, para que el nombre de la columna vaya todo junto

In [ ]:
columnas = ["price", "reference_price"] #cambio de title del 0 por el 1 índice

for col in columnas:
    df_unido[col] = df_unido[col].apply(cambiar_float) #aplicamos la función a cada columna con el bucle de iteración

In [ ]:
def cambiar_float(dato):
	try:
		return float(dato.replace(",", "."))
	except:
		return np.nan

x = "2,3"

cambiar_float(x)

In [ ]:
#Cambiar tipo de valor de una columna.  
Modificarlo a tipo fecha: 
df[‘columna’] = pd.to_datetime(df[‘columna’])

In [ ]:
# Función para reemplazar comas por puntos y convertir a float
def corregir_decimales(df, columnas):
    for col in columnas:
        df[col] = df[col].astype(str).str.replace(",", ".", regex=False)  # Reemplazar comas por puntos
        df[col] = pd.to_numeric(df[col], errors="coerce")  # Convertir a float, coerce valores no numéricos a NaN
    return df


## Valores únicos en variables categóricas

In [ ]:
lista_categ = []

In [ ]:
# Vamos a ver que columnas tienen valores nulos
porcentajes_nulos = (df.isnull().sum()[df.isna().sum() > 0] / df.shape[0])*100

In [ ]:
# Datos estadísticos de la columnas numéricas
df.describe().T

In [ ]:
# Datos estadísticos de la columnas categóricas
df.describe(include = "object").T

In [ ]:
len(df)

In [ ]:
df_productos["name"].value_counts()

In [ ]:
# Verificar si hay filas duplicadas
duplicados = df.duplicated().sum()
df_productos[df_productos.duplicated(subset="product_id", keep=False)].sort_values("product_id")

In [ ]:
df_prod_no_dup.drop(columns=["description"], inplace=True)
df_prod_no_dup.head()

In [ ]:
df_unido = df_precios.merge(df_prod_no_dup, on= ["product_id"], how="left")
df_unido.head()

df_unido.drop(columns=["supermarket"], inplace=True)
df_unido.head()

df_unido.shape

In [ ]:
df_unido.info()

## Filtrado de datos

In [ ]:
'''Operadores de comparación: >, <, >=, <=, ==, != 
Crear  una  condición  y  aplicarla  a  columnas.  Se  pueden  aplicar  diferentes 
condiciones con & o |. '''
#1. Crear condición: 
condición = df[‘columna’] == ‘x’ 
#2. Aplicar condición 
df_nuevo = df[condición] 
#3. df_nuevo es igual pero con la columna de la condición modificada

isin(): seleccionar filas que contienen valores específicos en una columna. Un valor 
o lista de valores 
df[‘columna’].isin(valores) 
1. Crear filtro: filtro = [‘valor1’, ‘valor2’] 
2. Aplicar filtro df_nuevo = df[df[‘columna’].isin(filtro)] 
3. df_nuevo es igual pero con las filas que contienen los valores del filtro

between(): filtrar por un rango 
nuevo_df = df[df[‘columna’].between(inicio, fin, inclusive=both/left/right/neither)] 
• both: incluye los valores de inicio y fin 
• left: incluye inicio pero no fin 
• right: incluye fin pero no inicio 
• neither: no incluye ni inicio ni fin 

str.contains(): filtrar por palabras. Devuelve un booleano. 
df[‘columna’].str.contains(pat, case=True, na=nan, regex=True) 
• pat: patrón de texto a buscar 
• case: (op) True distingue mayúsculas y minúsculas 
• na=nan: (op) 
• regex: (op) True se interpreta como regex 

## Agrupación de columnas

Operaciones de agregación: 
.count(): número valores no nulos 
.describe(): resumen de los principales estadísticos 
.sum(): suma de todos los valores 
.mean(): media de los valores 
.median(): mediana. Ordenados de menor a mayor, valor que queda en la mitad 
.min(): valor mínimo 
.max(): valor máximo 
.std(): desviación estándar. Cuánto se desvían los valores de la media. 
.var(): varianza. Cuánto se desvían los valores de la media al cuadrado.


Sintaxis: 
Para una operación 
variable = df.groupby(columna)[columna_operacion].operacion 
Para varias operaciones 
variable = df.groupby(columna)[columna_operacion].agg([‘op1’, ‘op2’])

Métodos: 
.reset_index(): nuevo DataFrame del resultado con índice en 0 
df.groupby(columna)[columna_operacion].operacion(numeric_only=True):  aplica 
la operación a todas columnas numéricas 
variable_agrupacion.ngroups: grupos formados después de la agrupación

## Aplicación de funciones

Para aplicar una función a una columna. Se puede añadir a una nueva columna de 
o aplica a una columna ya existente para modificar el DataFrame 
Sintaxis: 
df[‘nueva_columna’] = df[‘columna’].apply(función) 
df[‘columna_modificar’] = df[‘columna_modificar’].apply(función)
Para aplicar una función que debe recibir dos parámetros (dos columnas del DF) se 
hace con lambda. 
Sintaxis: 
df[‘nueva_columna’] = df.apply(lambda x: función(x[‘col1’], x[‘col2’]), axis=1)

OTROS MÉTODOS DE LIMPIEZA 
 
Para  aplicar  una  transformación  o  reemplazo  de  los  valores  a  un  Serie  o 
DataFrame 
 
.map(): transformación de cada elemento de una Serie 
Pasos:  
1. Crear diccionario de mapeo, keys valores actuales, values valores por los 
que reemplazar 
diccionario = {0: "No", 1: "Si"} 
2. Aplicar diccionario a una columna: 
df[‘col’] = df[‘col’].map(diccionario) 
 


 
.replace(): reemplaza valores en un DataFrame o Serie por otros especificados 
Sintaxis: 
df[‘col’] = df[‘col’].replace(valor a reemplazar, nuevo valor) 
Se utiliza para reemplazar un valor concreto de esa columna, no todos 

# Fase 2: Visualización